### **Import dependencies**

In [1]:
import collections
import tensorflow as tf
import federated.tensorflow_federated as tff
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np


2023-05-29 00:14:02.506823: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-29 00:14:02.745667: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-29 00:14:02.748434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 00:14:03.801884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
df = pd.read_csv('csv_files/merged_df_with_dates.csv')
attack_type = 'no_attack'

X = df.drop('is_malware').select_dtypes(include=['int', 'float']).values
y = df['is_malware']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into Tensors
X_train = tf.constant(X_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.int32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.int32)

# Function to create a neural network
def create_nn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the TFF version of the model
def model_fn():
    keras_model = create_nn()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_dataset.element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

# Simulate the clients' data
NUM_CLIENTS = 10
client_data = collections.OrderedDict()
for i in range(NUM_CLIENTS):
    start = i * len(X_train) // NUM_CLIENTS
    end = (i + 1) * len(X_train) // NUM_CLIENTS
    client_data[str(i)] = tf.data.Dataset.from_tensor_slices((X_train[start:end], y_train[start:end])).batch(1)

# Create the federated data
federated_data = [client_data[str(i)] for i in range(NUM_CLIENTS)]

# Create the TFF model and federated learning process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

